# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [3]:
# created table in workbench with below code

"""
CREATE TABLE rentals_may (
 rental_id INT(11) UNIQUE NOT NULL,
 rental_date DATETIME,
 inventory_id INT(11) DEFAULT NULL,
 customer_id INT(11) DEFAULT NULL,
 return_date DATETIME,
 staff_id INT(11) DEFAULT NULL,
 last_update DATETIME,
 CONSTRAINT PRIMARY KEY (rental_id)
 );
 """

'\nCREATE TABLE rentals_may (\n rental_id INT(11) UNIQUE NOT NULL,\n rental_date DATETIME,\n inventory_id INT(11) DEFAULT NULL,\n customer_id INT(11) DEFAULT NULL,\n return_date DATETIME,\n staff_id INT(11) DEFAULT NULL,\n last_update DATETIME,\n CONSTRAINT PRIMARY KEY (rental_id)\n );\n '

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [4]:
# inserted data into new table with below query in workbench 

"""
 INSERT INTO rentals_may
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 05;
 """

'\n INSERT INTO rentals_may\n SELECT *\n FROM rental\n WHERE MONTH(rental_date) = 05;\n '

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [5]:
# created table in workbench with below code

"""
CREATE TABLE rentals_june (
 rental_id INT(11) UNIQUE NOT NULL,
 rental_date DATETIME,
 inventory_id INT(11) DEFAULT NULL,
 customer_id INT(11) DEFAULT NULL,
 return_date DATETIME,
 staff_id INT(11) DEFAULT NULL,
 last_update DATETIME,
 CONSTRAINT PRIMARY KEY (rental_id)
 );
 """

'\nCREATE TABLE rentals_june (\n rental_id INT(11) UNIQUE NOT NULL,\n rental_date DATETIME,\n inventory_id INT(11) DEFAULT NULL,\n customer_id INT(11) DEFAULT NULL,\n return_date DATETIME,\n staff_id INT(11) DEFAULT NULL,\n last_update DATETIME,\n CONSTRAINT PRIMARY KEY (rental_id)\n );\n '

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [6]:
"""
 INSERT INTO rentals_june
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 06;
 """

'\n INSERT INTO rentals_june\n SELECT *\n FROM rental\n WHERE MONTH(rental_date) = 06;\n '

## Check the number of rentals for each customer for May.


In [7]:
pd.read_sql_query('SELECT customer_id, count(rental_id) FROM rentals_may GROUP BY customer_id ORDER BY count(rental_id)', engine)

,customer_id,count(rental_id)
0,459,1
1,333,1
2,456,1
3,88,1
4,532,1
...,...,...
515,596,6
516,161,6
517,506,7
518,109,7


## Check the number of rentals for each customer for June.
 

In [8]:
pd.read_sql_query('SELECT customer_id, count(rental_id) FROM rentals_june GROUP BY customer_id ORDER BY count(rental_id)', engine)

,customer_id,count(rental_id)
0,370,1
1,168,1
2,369,1
3,302,1
4,394,1
...,...,...
585,267,9
586,457,9
587,213,9
588,454,10


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [9]:
n_cust_may = pd.read_sql_query('SELECT customer_id, count(rental_id) as n_rentals FROM rentals_may GROUP BY customer_id ORDER BY count(rental_id)', engine)
n_cust_may.head()

,customer_id,n_rentals
0,459,1
1,333,1
2,456,1
3,88,1
4,532,1


In [10]:
n_cust_jun = pd.read_sql_query('SELECT customer_id, count(rental_id) as n_rentals FROM rentals_june GROUP BY customer_id ORDER BY count(rental_id)', engine)
n_cust_jun.head()

,customer_id,n_rentals
0,370,1
1,168,1
2,369,1
3,302,1
4,394,1


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [11]:
df_merged = n_cust_may.merge(n_cust_jun, left_on='customer_id', right_on='customer_id')
df_merged

,customer_id,n_rentals_x,n_rentals_y
0,459,1,7
1,333,1,4
2,456,1,7
3,88,1,3
4,532,1,2
...,...,...,...
507,596,6,2
508,161,6,2
509,506,7,5
510,109,7,5


In [12]:
def month_comparisson(df):
    for i in range(len(df['customer_id'])):
        df['difference'] = int(df['n_rentals_x'][i])-int(df['n_rentals_y'][i])
        return df['difference']

In [13]:
month_comparisson(df_merged)

0     -6
1     -6
2     -6
3     -6
4     -6
      ..
507   -6
508   -6
509   -6
510   -6
511   -6
Name: difference, Length: 512, dtype: int64

In [14]:
df_merged = df_merged.set_index('customer_id')

In [16]:
df_merged

,n_rentals_x,n_rentals_y,difference
customer_id,,,
459,1,7,-6
333,1,4,-6
456,1,7,-6
88,1,3,-6
532,1,2,-6
...,...,...,...
596,6,2,-6
161,6,2,-6
506,7,5,-6


In [19]:
def diff_id(id):
    return df_merged['difference'][id]

In [20]:
diff_id(459)

-6

In [21]:
def month_comparison(id):
    if diff_id(id)>0:
        return 'more rentals in May'
    elif diff_id(id)<0:
        return 'more rentals in June'
    else: 
        return 'no difference'

In [22]:
month_comparison(459)

'more rentals in June'